In [1]:
import numpy as np
import pandas as pd

from time import process_time
from tqdm.auto import tqdm

In [2]:
import requests
from bs4 import BeautifulSoup 
import json
import re

In [32]:
def get_web_page(url,headers,cook):
    w = requests.get(url,
                     headers=headers, 
                     cookies=cook)                                                      
    w.encoding = "utf-8"
    
    if w.status_code != 200 :
        print("Invalid url")
        return None
    else :
        return w.text #js檔 轉為字串用正則表達式處理

In [33]:
url = "https://referendum.2021.nat.gov.tw/pc/zh_TW/js/treeFF.js"
headers = {}
cook = {}

vote = get_web_page(url,headers,cook)

In [18]:
pattern = re.compile(r'(?<=firAreaID)\S+=(\S+);') # ()內是要的東西

city=[]
for i in range(len(pattern.findall(vote))):
    city.append(pattern.findall(vote)[i][1:-1])

In [19]:
area = []

for i in range(len(pattern.findall(vote))):
    pattern2 = re.compile(r'(?<=firAreaID\[{}\]\+\'00\'\+padding\(\')\d+'.format(i))
    area.append(pattern2.findall(vote))

In [20]:
numbers = []
n=-1
for i in city:
    n+=1
    for j in area[n]:
        numbers.append(i+"00"+j)

In [21]:
numbers

['6300000010',
 '6300000020',
 '6300000030',
 '6300000040',
 '6300000050',
 '6300000060',
 '6300000070',
 '6300000080',
 '6300000090',
 '6300000100',
 '6300000110',
 '6300000120',
 '6500000010',
 '6500000020',
 '6500000030',
 '6500000040',
 '6500000050',
 '6500000060',
 '6500000070',
 '6500000080',
 '6500000090',
 '6500000100',
 '6500000110',
 '6500000120',
 '6500000130',
 '6500000140',
 '6500000150',
 '6500000160',
 '6500000170',
 '6500000180',
 '6500000190',
 '6500000200',
 '6500000210',
 '6500000220',
 '6500000230',
 '6500000240',
 '6500000250',
 '6500000260',
 '6500000270',
 '6500000280',
 '6500000290',
 '6800000010',
 '6800000020',
 '6800000030',
 '6800000040',
 '6800000050',
 '6800000060',
 '6800000070',
 '6800000080',
 '6800000090',
 '6800000100',
 '6800000110',
 '6800000120',
 '6800000130',
 '6600000010',
 '6600000020',
 '6600000030',
 '6600000040',
 '6600000050',
 '6600000060',
 '6600000070',
 '6600000080',
 '6600000090',
 '6600000100',
 '6600000110',
 '6600000120',
 '66000001

In [22]:
def vote_data(rule):

    url = "https://referendum.2021.nat.gov.tw/pc/zh_TW/01/{}0000000.html".format(rule)
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}
    cook = {}


    w = requests.get(url,
                     headers=headers, #使用者資訊
                     cookies=cook) #資料
                                                                                                
    w.encoding = "utf-8"
    
    if w.status_code != 200 :
        print("Invalid url")
    else :
        soup = BeautifulSoup(w.text, "html") 

        pattern = re.compile(r'(?<=\xa0-\xa0)\w+')
        Adm = pattern.findall(soup.select('td[valign="bottom"]')[0].text)[0]

        pattern = re.compile(r'(?<=\xa0)\d+')
        vote = int(pattern.findall(soup.select('td[align="right"]')[0].text)[0])

        df = [i.text for i in soup.select('tr.trT td')]

        df.insert(0,Adm)
        df.append(vote)

        return df

In [23]:
Final_data=[]
for i in numbers:
    Final_data.append(vote_data(i))

In [24]:
final_data = pd.DataFrame(Final_data)
final_data.columns=["縣市行政區","同意票數","不同意票數","有效票數","無效票數","投票數","投票權人數","已完成投票所投票率(%)","有效同意票數對投票權人數(%)","已完成投票所"]

In [25]:
final_data

,縣市行政區,同意票數,不同意票數,有效票數,無效票數,投票數,投票權人數,已完成投票所投票率(%),有效同意票數對投票權人數(%),已完成投票所
0,臺北市松山區,"42,583","37,246","79,829",743,"80,572","162,964",49.44%,26.13%,134
1,臺北市信義區,"45,522","38,628","84,150",843,"84,993","181,747",46.76%,25.05%,166
2,臺北市大安區,"64,621","52,374","116,995","1,132","118,127","242,869",48.64%,26.61%,194
3,臺北市中山區,"41,082","45,181","86,263",794,"87,057","188,193",46.26%,21.83%,144
4,臺北市中正區,"31,561","27,558","59,119",584,"59,703","123,767",48.24%,25.5%,115
...,...,...,...,...,...,...,...,...,...,...
363,新竹市東區,"42,096","34,243","76,339",891,"77,230","169,450",45.58%,24.84%,160
364,新竹市北區,"27,138","25,882","53,020",657,"53,677","125,506",42.77%,21.62%,122
365,新竹市香山區,"12,213","13,399","25,612",353,"25,965","66,397",39.11%,18.39%,64
366,嘉義市東區,"19,300","23,106","42,406",311,"42,717","100,157",42.65%,19.27%,89
